In [ ]:
import torch
from sklearn.metrics import f1_score
from graph_qa_dataset import GraphQADataset
from torch_geometric.loader import DataListLoader
from model.graph_qa_model import GAT, HGT

In [ ]:
train_dataset = GraphQADataset(split="train", data_size=20)
val_dataset = GraphQADataset(split="validation", data_size=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = HGT(hidden_channels=64, out_channels=2, num_heads=4, num_layers=2, metadata=train_dataset.metadata)
model = model.to(device)

In [ ]:
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [ ]:
train_dataloader = DataListLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataListLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
def train(dataloader):
    model.train()
    
    total_loss = 0
    for batch in dataloader:
        data = batch[0]
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)

        loss = loss_op(out, data["context"].y)
        print(loss)
        total_loss += loss.item()
        loss.backward()

train(train_dataloader)

In [1]:
import numpy as np

In [3]:
[["train", [i[0], i[-1]]] for i in np.array_split(range(10), 3)]

[['train', [0, 3]], ['train', [4, 6]], ['train', [7, 9]]]